# 訓練模組

In [1]:
import os
import cv2
import math
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow.keras.backend as K

ModuleNotFoundError: No module named 'absl'

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
def on_train(point, batch_size, total):
    batch_size -= 1 # Inclube start point  
    if(point+batch_size>total):
        f = point+batch_size-total
        g = point+batch_size+1-f
        return list(range(point,g))+list(range(0, f))
    else:
        g = point+batch_size+1
        return list(range(point,g))

In [ ]:
def flag(point, batch_size, data_size):
    #Choose random indice for later picking.
    rnd_ind = on_train(point, batch_size, data_size-1)
    if((point + batch_size) >= data_size):
        point = point + batch_size - data_size
    else:
        point = point + batch_size 
    return  rnd_ind, point   

In [ ]:
def padding(img_np, mode, size):
    # 變化數 (C 4取0)+(C 4取1)+(C 4取2)+(C 4取3)+(C 4取4)
    s = img_np.shape
    if mode=="left":
        width = s[1]+size
        height = s[0]
        s1 = np.random.rand(height, width)
        s1[0:height, size:width] = img_np
    if mode=="right":
        width = s[1]+size
        height = s[0]
        s1 = np.random.rand(height, width)
        s1[0:height, 0:s[1]] = img_np
    if mode=="top":
        width = s[1]
        height = s[0]+size
        s1 = np.random.rand(height, width)
        s1[size:height, 0:width] = img_np
    if mode=="down":
        width = s[1]
        height = s[0]+size
        s1 = np.random.rand(height, width)
        s1[0:s[0], 0:width] = img_np
    img_np = cv2.resize(s1, s)
    return img_np

def padding_img(img_np, mode, size):
    # 變化數 (C 4取0)+(C 4取1)+(C 4取2)+(C 4取3)+(C 4取4)
    s = img_np.shape
    if mode=="left":
        width = s[1]+size
        height = s[0]
        s1 = np.random.rand(height, width,3)
        s1[0:height, size:width] = img_np
    if mode=="right":
        width = s[1]+size
        height = s[0]
        s1 = np.random.rand(height, width,3)
        s1[0:height, 0:s[1]] = img_np
    if mode=="top":
        width = s[1]
        height = s[0]+size
        s1 = np.random.rand(height, width,3)
        s1[size:height, 0:width] = img_np
    if mode=="down":
        width = s[1]
        height = s[0]+size
        s1 = np.random.rand(height, width,3)
        s1[0:s[0], 0:width] = img_np
    img_np = cv2.resize(s1, (s[0],s[1]))
    return img_np

In [ ]:
case = {0:"RGB",1:"RBG",2:"BGR",3:"BRG",4:"GRB",5:"GBR"}

In [ ]:
def color(img_np_, mode):
    img_np = img_np_.copy()
    a = img_np_
    # img_np is BGR cv2 default mode.
    # 6!，R、G、B三種元素，6階、6種可能。
    if  mode=="RGB":
        img_np[:,:,0:1] = a[:,:,2:3] #R
        img_np[:,:,1:2] = a[:,:,1:2] #G
        img_np[:,:,2:3] = a[:,:,0:1] #B
    elif mode=="RBG":
        img_np[:,:,0:1] = a[:,:,2:3] #R
        img_np[:,:,1:2] = a[:,:,0:1] #B
        img_np[:,:,2:3] = a[:,:,1:2] #G
    elif mode=="BGR":
        img_np[:,:,0:1] = a[:,:,0:1] #B
        img_np[:,:,1:2] = a[:,:,1:2] #G
        img_np[:,:,2:3] = a[:,:,2:3] #R
    elif mode=="BRG":
        img_np[:,:,0:1] = a[:,:,0:1] #B
        img_np[:,:,1:2] = a[:,:,2:3] #R
        img_np[:,:,2:3] = a[:,:,1:2] #G
    elif mode=="GRB":
        img_np[:,:,0:1] = a[:,:,1:2] #G
        img_np[:,:,1:2] = a[:,:,2:3] #R
        img_np[:,:,2:3] = a[:,:,0:1] #B
    elif mode=="GBR":
        img_np[:,:,0:1] = a[:,:,1:2] #G
        img_np[:,:,1:2] = a[:,:,0:1] #B
        img_np[:,:,2:3] = a[:,:,2:3] #R
    return img_np

In [ ]:
train_point = 0
val_point = 0

In [ ]:
input_size = [256, 256, 3]

def data_generator(isTrain, images_path, masks_path, image_ids, mask_ids, batch_size, img_size=input_size):
    global train_point, val_point
    train_point = 0
    val_point = 0
    '''
    images_path/masks_path: Images/Masks folder directory.
    images_ids/mask_ids: Ids for '.jpg' images/masks.
    img_size: Generated imgs/masks size.
    
    returns: batch of randomly-selected car&mask images value-scaled (0 -> 1). 
    '''
    data_size = len(image_ids)
    while True:
        #Choose random indice for later picking.        
        if isTrain:
            rnd_ind, point = flag(train_point, batch_size, data_size)
            train_point = point
        else:
            rnd_ind, point = flag(val_point, batch_size, data_size)
            val_point = point
        imgs = []
        masks = []
        for i in rnd_ind:
            #Pick a random id for car&mask images.
            img_id, mask_id = image_ids[i], mask_ids[i]
            #Load/resize images.
            # print(images_path +"\\" + img_id)
            img = cv2.imread(images_path +"\\" + img_id)
            mask = cv2.imread(masks_path +"\\" + mask_id, cv2.IMREAD_GRAYSCALE)            
            #Add to the batch data.
            # 隨機調整圖片顏色 2022/5/15 20:21
            ctr = np.random.randint(0,6)
            img = color(img, case[ctr])
            # 用隨機數判斷是否透視圖片，增強訓練集 (幅度30%以內) add on 2022/5/7 13:04
            ctr = np.random.randint(0,10)
            if ctr<=3:
                Perspective = True
                scale = 0.3/2
                s1 = int(img_size[1]*scale)
                s2 = int(img_size[0]*scale)
                x1 = np.random.randint(-s1,s1)
                x2 = np.random.randint(-s1,s1)
                x3 = img_size[1]-np.random.randint(-s1,s1)
                x4 = img_size[1]-np.random.randint(-s1,s1)
                y1 = np.random.randint(-s2,s2)
                y2 = img_size[0]-np.random.randint(-s2,s2)
                y3 = img_size[0]-np.random.randint(-s2,s2)
                y4 = np.random.randint(-s2,s2)
                pts1 = np.float32([[0,0], [0, img_size[0]], [img_size[1], img_size[0]], [img_size[1], 0]])
                pts2 = np.float32([[x1,y1], [x2, y2], [x3, y3], [x4, y4]])
                M = cv2.getPerspectiveTransform(pts1,pts2)
                img = cv2.warpPerspective(img,M,(img_size[1],img_size[0]))
                mask = cv2.warpPerspective(mask,M,(img_size[1],img_size[0]))
            # 用隨機數判斷是否旋轉圖片，增強訓練集
            ctr = np.random.randint(0,2)
            if ctr==1: # add on 2022/4/30
                inf = (int(img_size[0]/2), int(img_size[1]/2))
                inf1 = (int(img_size[0]), int(img_size[1]))
                M = cv2.getRotationMatrix2D(inf, np.random.randint(0,361), 1.0)
                img = cv2.warpAffine(img, M, inf1)
                mask = cv2.warpAffine(mask, M, inf1)
            ctr = np.random.randint(0,3)
            if ctr==1:
                # 水平翻轉
                img = cv2.flip(img, 1)
                mask = cv2.flip(mask, 1)
            elif ctr==2:
                # 上下翻轉
                img = cv2.flip(img, 0)
                mask = cv2.flip(mask, 0)
            # 用隨機數判斷是否對圖片做裁切 ( 幅度50%以內 )
            scale = 0.5/2
            ctr = np.random.randint(1,3)
            if ctr==1: # add on 2022/4/30
                ymin = 0 + np.random.randint(0,int(img_size[0]*scale))
                ymax = img_size[1] - np.random.randint(0,int(img_size[0]*scale))
                xmin = 0 + np.random.randint(0,int(img_size[1]*scale))
                xmax = img_size[0] - np.random.randint(0,int(img_size[1]*scale))
                img = img[ymin:ymax, xmin:xmax]
                mask = mask[ymin:ymax, xmin:xmax] 
                img = cv2.resize(img,(img_size[0], img_size[1]))
                mask = cv2.resize(mask,(img_size[0], img_size[1]))
            elif ctr==2: # add on 2022/5/7
                # 對圖片做填充 ( 幅度60%以內 )
                scale = 0.6/2
                s = np.random.randint(0, int(img_size[0]*scale))
                img = padding_img(img, "top", s)
                mask = padding(mask, "top", s)
                s = np.random.randint(0, int(img_size[0]*scale))
                img = padding_img(img, "down", s)
                mask = padding(mask, "down", s)
                s = np.random.randint(0, int(img_size[1]*scale))
                img = padding_img(img, "left", s)
                mask = padding(mask, "left", s)
                s = np.random.randint(0, int(img_size[1]*scale))
                img = padding_img(img, "right", s)
                mask = padding(mask, "right", s)
            # 對圖片做亮度、對比度處理，增強訓練集
            # 參考網站 https://www.wongwonggoods.com/python/python_opencv/opencv-modify-contrast/
            ctr = np.random.randint(1,101)
            if ctr>=35: # 65% 機率調整對比度
                brightness = 0
                contrast = np.random.randint(1,31) # - 減少對比度/+ 增加對比度 Max range(1,101)
                if ctr>=67.5:
                    contrast = contrast*-1
                B = brightness / 255.0
                c = contrast / 255.0 
                k = math.tan((45 + 44 * c) / 180 * math.pi)                
                img = (img - 127.5 * (1 - B)) * k + 127.5 * (1 + B)
                # 所有值必須介於 0~255 之間，超過255 = 255，小於 0 = 0
                img = np.clip(img, 0, 255)  
            ctr = np.random.randint(1,101)    
            if ctr>=35: # 65% 機率調整亮度
                phi = np.random.randint(5,10)/10 # phi>1 減少亮度  phi<1 增加亮度 phi:0.5~1.0 Max range(0.5~1.5)
                img = (img/255)**phi
                img = np.clip(img*255, 0, 255)  
            # ctr = np.random.randint(0,2)
            # if ctr==1 : # 50% 機率黑白顛倒
            #     img = 255-img
            img = img.reshape(img_size[0], img_size[1], 3)    
            mask = mask.reshape(img_size[0], img_size[1], 1)                
            imgs.append(img)
            masks.append(mask)   
        yield np.array(imgs, dtype=np.float16) / 255., np.array(masks, dtype=np.float16) / 255.

In [ ]:
def dice_coef(y_true, y_pred):
    '''
    Metric
    '''
    smooth = 1.
    # y_true = tf.where(y_true>0.5,1,0)
    # y_true = tf.cast(y_true,dtype=tf.float32)
    # y_pred = tf.where(y_pred>0.5,1,0)
    # y_pred = tf.cast(y_pred,dtype=tf.float32)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    '''
    Loss function
    '''
    loss = 1 - dice_coef(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    '''
    Mixed crossentropy and dice loss.
    '''
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

In [ ]:
def remove_desktop_ini(lst):
    i = 0
    for _ in lst:
        if _=="desktop.ini":
            lst.pop(i)
            break
        i += 1    

In [ ]:
imgPath = r"C:\Users\sky66\fiftyone\coco-2017\raw\img"
maskPath = r"C:\Users\sky66\fiftyone\coco-2017\raw\mask"

# 放mask
masks = list(os.listdir(maskPath))

# 放新圖片
images = list(os.listdir(imgPath))

remove_desktop_ini(masks)
remove_desktop_ini(images)

masks = sorted(masks)
images = sorted(images)

train = int(len(masks)*0.8)
val = len(images)-train

train_img = images[0:train]
train_mask = masks[0:train]

val_img = images[train:]
val_mask = masks[train:]

In [ ]:
# for e,r in zip(train_img, train_mask):
#     print(e, " ",r)

In [ ]:
# for e,r in zip(val_img, val_mask):
#     print(e, " ",r)

In [ ]:
print(len(images))
print(len(masks))

In [ ]:
modelPath = r'C:\Users\sky66\Desktop\my_model'

uNet = tf.keras.models.load_model(modelPath, custom_objects={"dice_coef":dice_coef, "dice_loss":dice_loss, "bce_dice_loss":bce_dice_loss})

In [ ]:
# modelPath1 = r'C:\Users\sky66\Desktop\my_model_gray'

# uNet1 = tf.keras.models.load_model(modelPath, custom_objects={"dice_coef":dice_coef, "dice_loss":dice_loss, "bce_dice_loss":bce_dice_loss})

# uNet.set_weights(uNet1.get_weights())

In [ ]:
#Prepare callbacks
patience = 100
LR_callback = ReduceLROnPlateau(monitor='val_loss', patience=patience, verbose=10, factor=0.5, min_lr=1e-030)
EarlyStop_callback = EarlyStopping(monitor='val_loss',patience=100, restore_best_weights=True)

In [ ]:
#Perpare data generators.
batch_size = 5
train_gen = data_generator(True, imgPath, maskPath, train_img, train_mask, batch_size=batch_size)
val_gen = data_generator(False, imgPath, maskPath, val_img, val_mask, batch_size=batch_size)

In [ ]:
# a=next(train_gen)
# plt.imshow(a[0][0].astype("float32"))

In [ ]:
# 1e-05 5e-06 2.5e-06 1.25e-06 6.25e-07 3.125e-07 1.5625e-07
lr = 5e-06
# uNet.compile(optimizer=Adam(learning_rate=lr), loss=bce_dice_loss, metrics=[dice_coef])
uNet.compile(optimizer=RMSprop(learning_rate=lr), loss=bce_dice_loss, metrics=[dice_coef])

In [ ]:
for _ in range(2):
    history = uNet.fit(train_gen, steps_per_epoch=int(train/batch_size),
                                 epochs=30, validation_data=val_gen,
                                 validation_steps=int(val/batch_size),
                                 callbacks=[LR_callback, EarlyStop_callback])
    uNet.save(modelPath)
    time.sleep(30*60)

In [ ]:
history = uNet.fit(train_gen, steps_per_epoch=int(train/batch_size),
                             epochs=30, validation_data=val_gen,
                             validation_steps=int(val/batch_size),
                             callbacks=[LR_callback, EarlyStop_callback])
uNet.save(modelPath)

In [ ]:
uNet.save(modelPath)

In [ ]:
# ResourceExhaustedError: Graph execution error: RAM不足，batch size太大

In [ ]:
x=list(range(len(history.history['dice_coef'])))
txt = "Unet Model\ndate:2022-05-07-03:48 a.m.\nlearning-rate: 1e-05\noptimizer:RMSprop"

In [ ]:
plt.plot(x,history.history['dice_coef'],label="dice_coef")
plt.plot(x,history.history['val_dice_coef'],label="val_dice_coef")

plt.title(txt,fontsize=15)
plt.xlabel("Epochs",fontsize=13)
plt.ylabel("Y",fontsize=13)
plt.legend()
plt.show()

In [ ]:
plt.plot(x,history.history['loss'],label="loss")
plt.plot(x,history.history['val_loss'],label="val_loss")

plt.title(txt,fontsize=15)
plt.xlabel("Epochs",fontsize=13)
plt.ylabel("Y",fontsize=13)
plt.legend()
plt.show()

In [ ]:
type(1.25e-06)

In [ ]:
"%.10f"%(1.25e-06)

# 進行偵測

In [ ]:
def phi(img, contrast): # contrast - 減少對比度/+ 增加對比度     
    # 調整對比度
    brightness = 0
    B = brightness / 255.0
    c = contrast / 255.0 
    k = math.tan((45 + 44 * c) / 180 * math.pi)                
    img = (img - 127.5 * (1 - B)) * k + 127.5 * (1 + B)
    # 所有值必須介於 0~255 之間，超過255 = 255，小於 0 = 0
    img = np.clip(img, 0, 255)    
    return img

def cv2_to_plt(img_np):
    img_np = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
    img_np = img_np/255
    return img_np

def plt_to_cv2(img_np):
    return cv2.cvtColor((img_np*255).astype("uint8"), cv2.COLOR_RGB2BGR)
    
def pred(img_path):    
    # load
    img_np = cv2.imread(img_path)
    img = cv2.imread(img_path)
    width = img.shape[1]
    height = img.shape[0]
    img_np = cv2.resize(img_np, (256,256))
    
    # use to predict
    # img_np = 255-img_np
    # print(img_np.shape)
    # img_np = phi(img_np, 100)
    img_np = img_np/255
    img_np = np.reshape(img_np,(256,256,3))
    img_np = np.expand_dims(img_np, 0)
    
    # predict
    pred_mask = uNet.predict(img_np)
    pred_mask = pred_mask[0]
    
    # process image
    # mask is GRAYSCALE
    pred_mask = pred_mask*255
    pred_mask = cv2.resize(pred_mask, (width,height))
    pred_mask = pred_mask/255
    pred_mask = np.reshape(pred_mask,(height,width,1))
    img = cv2_to_plt(img)
      
    return pred_mask*255, plt_to_cv2(img), plt_to_cv2(img*pred_mask)

def view(img_path):
    pred_mask, img, res = pred(img_path)
    img = cv2_to_plt(img)
    pred_mask /= 255
    # view
    fig, ax = plt.subplots(1, 3, figsize=(80,80))
    ax[0].imshow(img)
    ax[0].axis('off')
    ax[1].imshow(pred_mask, cmap='gray')
    ax[1].axis('off')
    # ax[2].imshow(pred_img*np.where(pred_mask>0.1, 1, 0))
    ax[2].imshow(img*pred_mask)
    ax[2].axis('off')
    plt.show()      
    
def stored(path, store = r"C:\Users\sky66\Desktop\predict"):
    name = path.split("\\")[-1].split(".")[0]
    a,b,c = pred(path)
    cv2.imwrite(store+"\\"+name+"_image.jpg", b)
    cv2.imwrite(store+"\\"+name+"_mask.jpg", a)   
    cv2.imwrite(store+"\\"+name+"_predict.jpg", c)        

In [ ]:
path = r"C:\Users\sky66\Desktop\Model\predict"+"\\"
for i in os.listdir(path):
    if i.split(".")[0].split("_")[-1] != "image":
        continue
    f = path+i
    view(f)
    input("Press Enter")

In [ ]:
a = next(train_gen)

In [ ]:
b = a[0][3]

In [ ]:
plt.imshow(b.astype("float32"))

In [ ]:
b = np.array([b], dtype=np.float16)

In [ ]:
plt.imshow(a[1][3],cmap="gray")

In [ ]:
c = uNet.predict(b)

In [ ]:
c.shape

In [ ]:
plt.imshow(c[0],cmap="gray")

In [ ]:
plt.imshow(c[0]*b[0])

In [ ]:
path = r"C:\Users\sky66\fiftyone\coco-2017\raw\train2017"+"\\"
for i in os.listdir(path):
    f = path+i
    view(f)
    input("Press Enter")

In [ ]:
cv2.imread("C:\\Users\\sky66\\fiftyone\\coco-2017\\mask\\0013ea2087020901 - 複製.png")

In [ ]:
load_img("C:\\Users\\sky66\\fiftyone\\coco-2017\\mask\\0013ea2087020901 - 複製.png")

In [ ]:
list(os.listdir("C:\\Users\\sky66\\fiftyone\\coco-2017\\mask"))

In [ ]:
# for i in images:
#     f = imgPath+"\\"+i
#     f = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
#     if f.shape!=(256,256):
#         print(i, f.shape)
#         f = cv2.resize(f,(256,256))
#         cv2.imwrite(imgPath+"\\"+i, f)

In [ ]:
# for i in masks:
#     f = maskPath+"\\"+i
#     f = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
#     if f.shape!=(256,256):
#         print(i, f.shape)
#         f = cv2.resize(f,(256,256))
#         cv2.imwrite(maskPath+"\\"+i, f)